# Update GG file to add additional parcels with GG / TRA_3 status near Redwood City Ferry Terminal

We have a list of parcels from Mark:
* we load parcel classes
* filter on the new subset
* update the relevant columns.
* check that no new GG categories are created (which would impact zoningmods)
* write updated file to disk

In [1]:
import pathlib
import pandas as pd
import geopandas as gpd
import os

In [2]:

HOME_DIR = pathlib.Path.home()
# in my (AO) case, M:\urban_modeling is mounted to /Volumes/Data/Models/urban_modeling
m_drive = M_DRIVE = pathlib.Path(
    "/Volumes/Data/Models") if os.name != "nt" else "M:/"

BOX_DIR = HOME_DIR / 'Library/CloudStorage/Box-Box'
HOME_DIR

PosixPath('/Users/aolsen')

In [3]:
PARCEL_GEOS_FILE_FBP_FEB25 = m_drive / \
    'urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/fbp_urbansim_parcel_classes_ot50pct_feb25_2025.csv'

PARCEL_GEOS_FILE_FBP_FEB14 = m_drive / \
    'urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/fbp_urbansim_parcel_classes_ot50pct_feb14_2025.csv'

PARCEL_GEOS_FILE_FBP_FEB25_rwc = m_drive / \
    'urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/fbp_urbansim_parcel_classes_ot50pct_feb25_rwc_update_2025.csv'

In [4]:
# convenience function for adding zoningmodcat as a function of the relevant components

def add_zoningmodcat(gg):

    parcels_geography_fbp_cols = ["gg_id", "exd_id", "pda_id",
                                  "tra_id", "hra_id",  "dis_id", "ppa_id", "ugb_id"]

    parcels_geographies_FBP[parcels_geography_fbp_cols] = parcels_geographies_FBP[parcels_geography_fbp_cols].astype(str).map(lambda x: x.lower()).fillna(
        'Remainder')

    zoning_mod_cols = ["gg_id", "exd_id",
                       "tra_id", "hra_id", "ppa_id", "ugb_id"]
    parcels_geographies_FBP["zoningmodcat"] = (
        parcels_geographies_FBP[zoning_mod_cols]
        .astype(str)
        .apply(lambda x: "".join(x), axis=1)
        .str.lower()
    )
    return parcels_geographies_FBP

# Load data

## Parcel classes

In [5]:
parcels_geographies_FBP = pd.read_csv(
    PARCEL_GEOS_FILE_FBP_FEB25)

parcels_geographies_FBP = parcels_geographies_FBP.set_index('parcel_id')

In [6]:
feb25 = add_zoningmodcat(PARCEL_GEOS_FILE_FBP_FEB25)

In [7]:
feb25.head()

,gg_id,pda_id,ppa_id,tra_id,hra_id,ugb_id,tpp_id,dis_id,exp2020_id,exsfd_id,exd_id,zoningmodcat
parcel_id,,,,,,,,,,,,
22.0,gg,oakland - west oakland,nan,nan,nan,ugb,brt3,nan,in,NaN,nan,ggnannannannanugb
58.0,gg,nan,ppa,nan,nan,ugb,NaN,nan,in,NaN,nan,ggnannannanppaugb
128.0,gg,oakland - west oakland,nan,nan,nan,ugb,bart1,nan,in,NaN,nan,ggnannannannanugb
217.0,gg,oakland - downtown & jack london square,nan,tra_4,nan,ugb,brt1,nan,in,NaN,nan,ggnantra_4nannanugb
358.0,gg,oakland - downtown & jack london square,nan,tra_4,nan,ugb,lrt2,dis,in,NaN,nan,ggnantra_4nannanugb


## Redwood City updates

In [8]:
updates_path = m_drive / \
    'urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/RWC_Parcels_to_add_to_tra5_gg.csv'
updates = pd.read_csv(updates_path)
print('Parcels in update: ',len(updates))
updates.head()

Parcels in update:  16


,OBJECTID *,Shape *,parcel_id,county,jurisdiction,acres,x,y,is_multipolygon,in_growth_geography_designation,in_high_resource_area,in_priority_dev_area,in_priority_prod_area,in_transit_rich_area,in_urban_growth_boundaries_area,Shape_Length,Shape_Area
0,1,Polygon,1173302,San Mateo,redwood_city,4.439077,-122.212159,37.504586,0,1,<Null>,<Null>,<Null>,tra5,1,758.417815,17964.334570
1,2,Polygon,1173388,San Mateo,redwood_city,1.995135,-122.201318,37.510768,0,1,<Null>,<Null>,<Null>,tra5,1,367.107992,8074.037571
2,3,Polygon,1173382,San Mateo,redwood_city,3.139200,-122.204709,37.511279,0,1,<Null>,<Null>,<Null>,tra5,1,478.591783,12703.911990
3,5,Polygon,1173373,San Mateo,redwood_city,4.367664,-122.203297,37.513483,0,1,<Null>,<Null>,<Null>,tra5,1,611.947558,17675.335420
4,7,Polygon,1173383,San Mateo,redwood_city,2.733019,-122.205753,37.510605,0,1,<Null>,<Null>,<Null>,tra5,1,434.422991,11060.153470


In [11]:
## check gg / tra status for the updates parcels before the update
# note zoningmodcat is nannannannannanugb

feb25.loc[updates.parcel_id]

,gg_id,pda_id,ppa_id,tra_id,hra_id,ugb_id,tpp_id,dis_id,exp2020_id,exsfd_id,exd_id,zoningmodcat
parcel_id,,,,,,,,,,,,
1173302.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173388.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173382.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173373.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173383.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173303.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173296.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173384.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb
1173385.0,nan,nan,nan,nan,nan,ugb,NaN,dis,in,NaN,nan,nannannannannanugb


# Apply updates to the parcel set - they should be tra_5 and gg

In [12]:
# apply updates

parcels_geographies_FBP.loc[updates.parcel_id,
                            ['gg_id', 'tra_id']] = ['gg', 'tra_5']

In [13]:
parcels_geographies_FBP.loc[updates.parcel_id,['gg_id','tra_id']]

,gg_id,tra_id
parcel_id,,
1173302.0,gg,tra_5
1173388.0,gg,tra_5
1173382.0,gg,tra_5
1173373.0,gg,tra_5
1173383.0,gg,tra_5
1173303.0,gg,tra_5
1173296.0,gg,tra_5
1173384.0,gg,tra_5
1173385.0,gg,tra_5


# Check what the zoningmodcat would be given the updates

In [14]:
# reapply zoningmodcat based on updated components
feb25_rc = add_zoningmodcat(PARCEL_GEOS_FILE_FBP_FEB25)

In [15]:
feb25_rc.loc[updates.parcel_id]

,gg_id,pda_id,ppa_id,tra_id,hra_id,ugb_id,tpp_id,dis_id,exp2020_id,exsfd_id,exd_id,zoningmodcat
parcel_id,,,,,,,,,,,,
1173302.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173388.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173382.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173373.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173383.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173303.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173296.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173384.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173385.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb


In [16]:
# this is the new zoningmodcat

derived_zoningmodcat = feb25_rc.loc[updates.parcel_id].zoningmodcat.unique()
derived_zoningmodcat 

array(['ggnantra_5nannanugb'], dtype=object)

In [23]:
# checking this zoningmodcat exists elsewhere in the file - checks out

parcels_geographies_FBP[parcels_geographies_FBP.zoningmodcat.isin(derived_zoningmodcat)].groupby('pda_id').size()

pda_id
alameda - naval air station                                                                       31
alameda - northern waterfront                                                                    133
berkeley - southside/telegraph avenue                                                            110
brentwood - brentwood transit village                                                             40
campbell - central campbell                                                                       17
campbell - hamilton avenue specific plan area                                                      9
concord - downtown                                                                                78
cotati - downtown and cotati depot                                                                23
daly city - bayshore                                                                              65
daly city - mission boulevard                                                       

# Write updated parcel classes table to disk

In [67]:
parcels_geographies_FBP.to_csv(PARCEL_GEOS_FILE_FBP_FEB25_rwc)

In [24]:
parcels_geographies_FBP.loc[updates.parcel_id]

,gg_id,pda_id,ppa_id,tra_id,hra_id,ugb_id,tpp_id,dis_id,exp2020_id,exsfd_id,exd_id,zoningmodcat
parcel_id,,,,,,,,,,,,
1173302.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173388.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173382.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173373.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173383.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173303.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173296.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173384.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
1173385.0,gg,nan,nan,tra_5,nan,ugb,NaN,dis,in,NaN,nan,ggnantra_5nannanugb
